# 課題3：タイタニック号乗客の生存状況の分類モデル作成

本課題では、`titanic` というデータセットを使います。これは、1912年に発生したタイタニック号の沈没事故における乗客の生存状況に関するデータセットです。元々は、[Encyclopedia Titanica](https://www.encyclopedia-titanica.org/)で掲載されたデータと言われており、このデータセットを組み込んだPythonのライブラリも複数あります。

今回は、`seaborn` のライブラリに組み込まれた `titanic` のデータセットを使います。各セルに入っているコメントの下に、実行するコードを記入してください。わからない場合は、ここまでのレッスン内容や各種ライブラリの公式ドキュメントを参照しましょう。

## 1. 必要なライブラリのimport

In [5]:
# 必要なライブラリのimport（変更しないでください）
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from sklearn.model_selection import train_test_split

# DataFrameですべての列を表示する設定（変更しないでください）
pd.options.display.max_columns = None

## 2. データの読み込み

seabornに添付のデータセットから「titanic」を読み込み、内容を確認します。

In [6]:
# seabornからtitanicのデータセットを読み込む（変更しないでください）
dataset = sns.load_dataset("titanic")

`sns.load_dataset()` で読み込んだデータは、pandasのDataFrameになっています。

In [7]:
# datasetの先頭5件を確認
dataset.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


### 使用する列の指定

今回は `survived, pclass, sex, age, sibsp, parch, fare, embarked` の列を使用します。

#### 参考:各列の説明

- `survived`: 生存区分（0:死亡, 1:生存）
- `pclass`: チケットクラス
- `sex`: 性別（male:男性, female:女性）
- `age`: 年齢
- `sibsp`: 同乗している兄弟や配偶者の数
- `parch`: 同乗している親や子供の数
- `fare`: 料金
- `embarked`: 乗船した港（頭文字）
- `class`: 客室クラス
- `who`: 性別（man:男性, woman:女性）
- `adult_male`: 成人男性ならTrue
- `deck`: 事故の際にどのデッキにいたか
- `embark_town`: 乗船した港名
- `alive`: 生存区分（no:死亡, yes:生存）
- `alone`: 1人で乗船したか

In [8]:
# datasetから「survived, pclass, sex, age, sibsp, parch, fare, embarked」の列を取得して
# datasetに代入（上書き）する
dataset = dataset[["survived", "pclass", "sex", "age", "sibsp", "parch", "fare", "embarked"]]


In [9]:
# 改めてdatasetの先頭5件を表示
dataset.head()


,survived,pclass,sex,age,sibsp,parch,fare,embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


## 3. データの前処理

### 要約統計量の表示

In [10]:
# 要約統計量を表示
dataset.describe()


,survived,pclass,age,sibsp,parch,fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


### 欠損値の確認と補完

In [11]:
# 各列の欠損値の数を確認
dataset.isnull().sum()


survived      0
pclass        0
sex           0
age         177
sibsp         0
parch         0
fare          0
embarked      2
dtype: int64

ageの欠損値は平均値で補完します。

In [12]:
# ageの欠損値を、ageの平均値で補完する
dataset["age"].fillna(dataset["age"].mean(), inplace=True)
dataset["age"].isnull().sum()


C:\Users\subek\AppData\Local\Temp\ipykernel_1336\2722331130.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset["age"].fillna(dataset["age"].mean(), inplace=True)


np.int64(0)

embarkedの欠損値は、もっとも乗船者数の多い港で補完します。

その方法はいくつかありますが、ここではその1つとして、DataFrameの特定の1列（Series）が持つ `value_counts()` メソッドを紹介します。このメソッドを実行すると、その列が持つ値ごとのデータ数がわかります。

参考：[pandas.Series.value_counts](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.value_counts.html)

In [13]:
# 乗船者数の多い港を value_counts メソッドで確認
dataset["embarked"].value_counts()

embarked
S    644
C    168
Q     77
Name: count, dtype: int64

`values_count()` の結果を見て、もっとも乗船者数の多い港の文字で欠損値を埋めるようにします。

In [14]:
# embarkedの欠損値をもっとも乗船者数の多い港にて補完
most_freq_port = dataset["embarked"].value_counts().idxmax()

dataset["embarked"].fillna(most_freq_port, inplace=True)



C:\Users\subek\AppData\Local\Temp\ipykernel_1336\2083994495.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset["embarked"].fillna(most_freq_port, inplace=True)


上記の処理により、欠損値がなくなったかを確認しましょう。

In [15]:
# 欠損値の数を確認し、補完後の欠損値が0であることを確認
dataset["embarked"].isnull().sum()

np.int64(0)

### ダミー変数への変換

sexとembarkedをダミー変数に変換します。

In [16]:
# datasetのsexとembarkedをダミー変数に変換してdataset2に代入する
dataset2 = pd.get_dummies(dataset, columns=["sex", "embarked"], drop_first=True)


In [17]:
# dataset2のデータの最初の5行を表示
dataset2.head()


,survived,pclass,age,sibsp,parch,fare,sex_male,embarked_Q,embarked_S
0,0,3,22.0,1,0,7.2500,True,False,True
1,1,1,38.0,1,0,71.2833,False,False,False
2,1,3,26.0,0,0,7.9250,False,False,True
3,1,1,35.0,1,0,53.1000,False,False,True
4,0,3,35.0,0,0,8.0500,True,False,True


## 4. 目的変数と説明変数の選択

ここでは、以下の列を使用します。

- 目的変数: `survived`
- 説明変数: それ以外

dataset2より目的変数と説明変数に該当する列を取得してnumpy配列に変換し、変数YとXに格納します。列の除外には、DataFrameの `drop` を使います。`データフレーム.drop(columns=除外したい列名)` です。

In [18]:
# Y:目的変数に該当する列
Y = dataset2["survived"].values

# X:説明変数に該当する列。dataset2からsurvivedを除外
X = dataset2.drop(columns=["survived"]).values


In [19]:
# YとXの形状を確認
print("Yの形状:", Y.shape)
print("Xの形状:", X.shape)

Yの形状: (891,)
Xの形状: (891, 8)


## 5. データの分割

この課題ではホールドアウト法でデータを分割します。

In [23]:
# X と Y を 機械学習用データとテストデータに7:3に分ける(X_train, X_test, Y_train, Y_test)
from sklearn.model_selection import train_test_split



In [24]:
# 機械学習用データを、学習データと検証データに7:3に分ける(X_train, X_valid, Y_train, Y_valid)
X_train, X_valid, Y_train, Y_valid = train_test_split(
    X_train, Y_train, test_size=0.3, random_state=42
)


In [25]:
# 形状を確認:X_train, X_valid, X_test, Y_train, Y_valid, Y_test
print("X_train:", X_train.shape)
print("X_valid:", X_valid.shape)
print("X_test :", X_test.shape)
print("Y_train:", Y_train.shape)
print("Y_valid:", Y_valid.shape)
print("Y_test :", Y_test.shape)

X_train: (436, 8)
X_valid: (187, 8)
X_test : (268, 8)
Y_train: (436,)
Y_valid: (187,)
Y_test : (268,)


## 6. モデルの選択

ロジスティック回帰と決定木、ランダムフォレスト、SVMの4つのモデルを作成して比較します。

In [26]:
# 必要なライブラリの追加import（変更しないでください）
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score

モデルの評価（性能の比較）には、F1値を使ってください。以下には1つだけセルを用意していますが、モデルを4つ作って比較する処理のためにセルを増やしてもかまいません。

In [27]:
# 4つのモデルを作成し、それぞれのF1値を出力する
models = {
    "LogisticRegression": LogisticRegression(max_iter=2000),
    "DecisionTree": DecisionTreeClassifier(random_state=42),
    "RandomForest": RandomForestClassifier(n_estimators=300, random_state=42),
    "SVM(RBF)": SVC(kernel="rbf")
}

## 7. パラメータのチューニング

GridSearchCVを使い、性能の良かったランダムフォレストのパラメータのチューニングを行ないます。パラメータの候補については、レッスン本編を参考にしてください。

In [28]:
# 必要なライブラリの追加import（変更しないでください）
from sklearn.model_selection import GridSearchCV

In [29]:
# 性能の良かったモデルを作成
best_model = RandomForestClassifier(random_state=42)


In [32]:
# パラメータの指定
best_model = RandomForestClassifier(
    n_estimators=300,     
    max_depth=10,        
    min_samples_split=5,  
    min_samples_leaf=2,   
    random_state=42
)


In [33]:
# グリッドサーチのオブジェクトを作成
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [38]:
# データの分割:機械学習用データを学習と検証に分けるのはクロスバリデーションで行ってくれる
# （Xg_train, Xg_test, Yg_train, Yg_test）

from sklearn.model_selection import train_test_split

Xg_train, Xg_test, Yg_train, Yg_test = train_test_split(
    X, Y, test_size=0.3, random_state=42
)

grid_search = GridSearchCV(
    estimator=best_model,
    param_grid=param_grid,
    scoring='f1',
    cv=5,
    n_jobs=-1,
    verbose=1
)

In [39]:
# グリッドサーチを実行する
grid_search.fit(Xg_train, Yg_train)


Fitting 5 folds for each of 81 candidates, totalling 405 fits


,estimator,RandomForestC...ndom_state=42)
,param_grid,"{'max_depth': [5, 10, ...], 'min_samples_leaf': [1, 2, ...], 'min_samples_split': [2, 5, ...], 'n_estimators': [100, 200, ...]}"
,scoring,'f1'
,n_jobs,-1
,refit,True
,cv,5
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,n_estimators,100


In [40]:
# 最適なパラメータを表示
print("最適なパラメータ:", grid_search.best_params_)
print("平均F1スコア（CV）:", grid_search.best_score_)

最適なパラメータ: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 100}
平均F1スコア（CV）: 0.7440205228690765


ここで得たパラメータをもとに、モデルを再度作成します。

In [41]:
# 最適なパラメータによるモデルの作成
best_params = grid_search.best_params_

best_model = RandomForestClassifier(
    **best_params,      # 最適なパラメータを展開して適用
    random_state=42     # 乱数固定（再現性確保）
)


# モデルの学習
best_model.fit(Xg_train, Yg_train)


# モデルの予測
Yg_pred = best_model.predict(Xg_test)


In [43]:
# F1値の出力
from sklearn.metrics import f1_score

f1 = f1_score(Yg_test, Yg_pred)


## 8. テストデータによる汎化性能の確認

最後にテストデータでモデルの汎化性能を確認しましょう。

In [44]:
# テストデータを使って予測を行いF1値を算出
print("テストデータでのF1スコア:", f1)

テストデータでのF1スコア: 0.7586206896551724
